In [ ]:
# for training
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install fuzzywuzzy python-Levenshtein
!pip install bitsandbytes==0.41.3
!pip install -q -U sentencepiece
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install nltk==3.5.0
!pip install bert-score
!pip install rouge
#for inference time
!pip install xlrd
!pip install PyPDF2
!pip install keybert
!pip install rarfile
!pip install openpyxl
!pip install python-pptx
!pip install python-docx
!pip install keybert[use]
!pip install keybert[spacy]
!pip install keybert[flair]
!pip install keybert[gensim]
!pip install sense2vec==2.0.1
!python -m spacy download en_core_web_sm
!pip install git+https://github.com/boudinfl/pke.git
!wget https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz
!tar -xvf  s2v_reddit_2015_md.tar.gz

In [ ]:
import os
import torch

os.environ["WANDB_DISABLED"] = "true"
if torch.cuda.is_available()==True :
    os.environ["CUDA_VISIBLE_DEVICES"]="0"

# #########################################

# For Inference Time

### dealing with the data

In [ ]:
import PyPDF2
import requests
from bs4 import BeautifulSoup
import csv
import zipfile
import json
import xml.etree.ElementTree as ET
import docx
import pptx
import rarfile
import xlrd  # For XLS files
import openpyxl  # For XLSX files
import shutil  # For extracting ZIP files

def read_csv(file_path):
    with open(file_path, 'r', newline='') as csvfile:
        csv_reader = csv.reader(csvfile)
        csv_data = [row for row in csv_reader]
    return csv_data

def read_text(file_path):
    with open(file_path, 'r') as f:
        text_data = f.read()
    return text_data

def read_pdf(file_path):
    pdf_file = open(file_path, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text_data = []
    for page in pdf_reader.pages:
        text_data.append(page.extract_text())
    return text_data

def read_web_page(url):
    result = requests.get(url)
    src = result.content
    soup = BeautifulSoup(src, 'html.parser')
    text_data = ''
    for p in soup.find_all('p'):
        text_data += p.get_text() + '\n'
    return text_data

def read_docx(file_path):
    doc = docx.Document(file_path)
    text_data = '\n'.join([paragraph.text for paragraph in doc.paragraphs])
    return text_data

def read_pptx(file_path):
    ppt = pptx.Presentation(file_path)
    text_data = ''
    for slide in ppt.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                text_data += shape.text + '\n'
    return text_data

def read_xlsx(file_path):
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active
    text_data = ''
    for row in sheet.iter_rows(values_only=True):
        text_data += ' '.join([str(cell) for cell in row if cell is not None]) + '\n'
    return text_data


def read_json(file_path):
    with open(file_path, 'r') as f:
        json_data = json.load(f)
    return json_data

def read_html(file_path):
    with open(file_path, 'r') as f:
        html_data = f.read()
    return html_data

def read_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    return ET.tostring(root, encoding='unicode')

def read_zip(file_path):
    file_contents = []
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        for file_info in zip_ref.infolist():
            with zip_ref.open(file_info) as file:
                # Call read_data to handle reading and processing the file contents
                file_data = read_data(file)
                file_contents.append(file_data)
    return file_contents

def read_rar(file_path):
    file_contents = []
    with rarfile.RarFile(file_path, 'r') as rar_ref:
        for rar_info in rar_ref.infolist():
            with rar_ref.open(rar_info) as file:
                # Call read_data to handle reading and processing the file contents
                file_data = read_data(file)
                file_contents.append(file_data)
    return file_contents


def read_data(file_path):
    # Check if the file is a CSV file
    if file_path.endswith('.csv'):
        return read_csv(file_path)

    # Check if the file is a text file
    elif file_path.endswith('.txt'):
        return read_text(file_path)

    # Check if the file is a PDF file
    elif file_path.endswith('.pdf'):
        return read_pdf(file_path)

    # Check if the file is a DOCX file
    elif file_path.endswith('.docx'):
        return read_docx(file_path)

    # Check if the file is a PPTX file
    elif file_path.endswith('.pptx'):
        return read_pptx(file_path)

    # Check if the file is an XLSX file
    elif file_path.endswith('.xlsx'):
        return read_xlsx(file_path)

    # Check if the file is a JSON file
    elif file_path.endswith('.json'):
        return read_json(file_path)

    # Check if the file is an HTML file
    elif file_path.endswith('.html'):
        return read_html(file_path)

    # Check if the file is an XML file
    elif file_path.endswith('.xml'):
        return read_xml(file_path)

    # Check if the file is a ZIP file
    elif file_path.endswith('.zip'):
        return read_zip(file_path)

    # Check if the file is a RAR file
    elif file_path.endswith('.rar'):
        return read_rar(file_path)

    # # Assume it's a web page if it's not a file
    # elif os.path.exists(file_path):
    #     return read_text(file_path)

    # Treat it as a web page if it's a URL
    elif file_path.startswith('http'):
        return read_web_page(file_path)

    # If the file type is unknown, return None
    else:
        print("Unsupported type")
        return None

In [ ]:
file_path=''
text=read_data(file_path)
text

### Cleaning the data

In [ ]:
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
stop_words = stopwords.words('english')
# arabic_stopwords = stopwords.words('arabic')

In [ ]:
def remove_non_ascii(text):
    """Remove non-ASCII characters from list of tokenized words"""
    return text.encode('ascii','ignore').decode()
    #return ''.join(char for char in text if char.isalpha() and char.isnumeric() or 'ARABIC' in unicodedata.name(char, ''))

def remove_brackets_num(text):
    return re.sub("\*?","",text)

def to_lowercase(text):
    return text.lower()

def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+','',text)

def remove_whitespace(text):
      return text.strip()

def remove_punctuation(text):
    punctuation= '''!()[]{};:'"\<>/?$%^&*_`~='''
    for punc in punctuation:
        text=text.replace(punc,"")
    return text

def remove_emails(text):
    return re.sub(r'[A-Za-z0-9]*@[A-Za-z]*\.?[A-Za-z0-9]*', "", text)

def text2words(text):
    return word_tokenize(text)

def remove_stopwords(words,stop_words):
    return [word for word in words if word not in stop_words]


def normalize_text(text):
    text = remove_non_ascii(text)
    text= remove_brackets_num(text)
    text = to_lowercase(text)
    #text=replace_numbers(text)
    text= remove_whitespace(text)
    text = remove_punctuation(text)
    text= remove_emails(text)
    words = text2words(text)
    #words = remove_stopwords(words, stop_words)

    return ' '.join(words)

In [ ]:
N_text=normalize_text(text)
N_text

### Generate KeyWords

In [ ]:
from keybert import KeyBERT
kw_tool = KeyBERT()

In [ ]:
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# def jaccard_similarity(word1, word2):
#     set1 = set(word1)
#     set2 = set(word2)

#     intersection = len(set1.intersection(set2))
#     union = len(set1.union(set2))

#     similarity = intersection / union if union > 0 else 0
#     return similarity

def calculate_similarity(sentence1, sentence2):
    # Initialize Porter Stemmer
    stemmer = PorterStemmer()

    # Tokenize and stem the sentences
    stemmed_sentence1 = ' '.join([stemmer.stem(word) for word in sentence1.split()])
    stemmed_sentence2 = ' '.join([stemmer.stem(word) for word in sentence2.split()])

    # Convert the stemmed sentences into vectors
    vectorizer = CountVectorizer().fit([stemmed_sentence1, stemmed_sentence2])
    vectorized_sentences = vectorizer.transform([stemmed_sentence1, stemmed_sentence2])

    # Calculate cosine similarity
    cosine_sim = cosine_similarity(vectorized_sentences)[0][1]

    return cosine_sim


def extract_keywords_from_text1(text):
    # Extract keywords from the given text
    KeyBERT1 = kw_tool.extract_keywords(text, keyphrase_ngram_range=(1,1), top_n=10)
    KeyBERT2 = kw_tool.extract_keywords(text, keyphrase_ngram_range=(2,2), top_n=10)

    # Combine all extracted keywords
    all_keywords = [key[0] for key in KeyBERT1] + \
                   [key[0] for key in KeyBERT2]
    # Filter out empty keywords
    all_keywords = [keyword for keyword in all_keywords if keyword]

    # Filter out very similar keywords
    similarity_threshold_between_keywords = 0.4  # Threshold for similarity between keywords
    unique_keywords = []
    for keyword in all_keywords:
        if all(calculate_similarity(keyword, existing_keyword) < similarity_threshold_between_keywords for existing_keyword in unique_keywords):
            unique_keywords.append(keyword)

    return unique_keywords

In [ ]:
unique_keywords1=extract_keywords_from_text1(N_text)
unique_keywords1

In [ ]:
import pke

def extract_keywords_from_text2(text):
    # Initialize keyphrase extraction model, here TopicRank
    extractor = pke.unsupervised.TopicRank()

    # Load the content of the document
    extractor.load_document(input=text, language='en')

    # Keyphrase candidate selection: in the case of TopicRank: sequences of nouns
    # and adjectives (i.e., `(Noun|Adj)*`)
    extractor.candidate_selection()

    # Candidate weighting: using a random walk algorithm
    extractor.candidate_weighting()

    # N-best selection, keyphrases contains the 10 highest scored candidates
    keyphrases = extractor.get_n_best(n=20)

    # Extract keyphrases
    keywords = [keyphrase for keyphrase, score in keyphrases]

    # Calculate similarity with keywords
    unique_keywords = []

    # Handling unigrams and bigrams separately
    unigrams = [keyphrase for keyphrase in keywords if len(keyphrase.split()) == 1]
    bigrams = [keyphrase for keyphrase in keywords if len(keyphrase.split()) == 2]

    # Add unigrams to unique_keywords directly
    unique_keywords.extend(unigrams)

    # Filter bigrams based on similarity with existing keywords
    for keyphrase in bigrams:
        similarity = calculate_similarity(keyphrase, ' '.join(keywords))
        if similarity < 0.4:  # Adjust the similarity threshold as needed
            unique_keywords.append(keyphrase)

    return unique_keywords

In [ ]:
unique_keywords2=extract_keywords_from_text2(N_text)
unique_keywords2

### Generate Distractors

In [ ]:
from sense2vec import Sense2Vec
# load sense2vec vectors
s2v = Sense2Vec().from_disk('s2v_old')

In [ ]:
from collections import OrderedDict

def sense2vec_get_words(word, s2v):
    output = []
    word = word.lower()

    sense = s2v.get_best_sense(word)
    similarity_threshold = 0.3
    out = []

    if sense is not None:
        most_similar = s2v.most_similar(sense, n=20)
        for sim in most_similar:
            append_word= sim[0].split("|")[0].replace("_", " ").lower()

            # Check similarity with keyword
            similarity_keyword = calculate_similarity(word, append_word)
            #print(f"Similarity between '{word}' and '{append_word}': {similarity_keyword}")

            # Check if similarity with keyword is above the threshold
            if similarity_keyword >= similarity_threshold:
                continue

            # Check similarity with existing distractors
            similarity_to_existing = [calculate_similarity(append_word, existing_distractor) for existing_distractor in output]

            # Check if similarity with any existing distractor is above the threshold
            if any(similarity >= similarity_threshold for similarity in similarity_to_existing):
                continue

            # If the conditions are met, append the word to the list of output
            output.append(append_word.title())

        out = list(OrderedDict.fromkeys(output))
    return out[:3]

In [ ]:
for word in unique_keywords:
    existing_distractor=sense2vec_get_words(word, s2v)
    print(word , existing_distractor)

### At Inference Time

In [ ]:
#general question generation model
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
HUGGING_FACE_USER_NAME=''
model_name=''
peft_model_id = f"{HUGGING_FACE_USER_NAME}/{model_name}"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=False, device_map='auto')
G_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
G_model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
def generate_questions(context, answer, distractors):
    device = next(G_model.parameters()).device
    input_text = f"Given the context '{context}' and the answer '{answer}' , what question can be asked?"
    encoding = G_tokenizer.encode_plus(input_text, padding=True,truncation=True, return_tensors="pt").to(device)

    output_tokens = G_model.generate(**encoding, early_stopping=True, num_beams=5, num_return_sequences=1, no_repeat_ngram_size=2, max_length=200)
    question = G_tokenizer.decode(output_tokens[0], skip_special_tokens=True).replace("question:", "").strip()
    return question

In [ ]:
context_printed = False  # Flag to keep track of whether the context has been printed

# Iterate over unique keywords and generate questions for keywords with distractor lists
for word in unique_keywords:
    existing_distractors = sense2vec_get_words(word, s2v)
    if existing_distractors:
        if not context_printed:
            print("Context:", N_text)
            print()
            context_printed = True  # Set the flag to True after printing the context

        context = N_text
        answer = word
        distractors = existing_distractors
        question = generate_questions(context, answer, existing_distractors)

        print("Answer:", answer)
        print("Distractors:", existing_distractors)
        print("Generated Question:", question)
        print()

In [ ]:
def process_and_generate_questions(file_path):
    # Read the data from the file
    example = read_data(file_path)
    N_text_file, keywords, keyword_question_distractors = "", [], []

    # Check if example is not None and not empty
    if example is not None and example.strip():
        # Remove empty lines between sentences
        cleaned_text = '\n'.join(line.strip() for line in example.split('\n') if line.strip())

        # Concatenate lines and separate them with a period
        concatenated_text = '.'.join(cleaned_text.split('\n'))

        # Tokenize the text using G_tokenizer
        tokens = G_tokenizer.tokenize(concatenated_text)

        # Check if the number of tokens is less than 1024
        if len(tokens) < 1024: # or 512 for google-flan-t5
            # Normalize the concatenated text
            N_text_file = normalize_text(concatenated_text)

            # Check if N_text_file has text
            if N_text_file:
                keywords = extract_keywords_from_text(N_text_file)
                if keywords:
                    for word in keywords:
                        current_distractors = sense2vec_get_words(word, s2v)
                        if current_distractors:
                            question = generate_questions(N_text_file, word)
                            keyword_question_distractors.append((word, current_distractors, question))
                else:
                    print("No keywords generated.")
            else:
                print("No text available.")
        else:
            print("The tokenized text has more than 1024 tokens.")
    else:
        print("The file is empty or large.")
        # Handle the case where the file is empty or cannot be loaded

    result = [(N_text_file, keyword_question_distractors)]

    return result

# Example usage:
file_path = ""
Total_List = process_and_generate_questions(file_path)

# Print the context, keyword, question, and distractors
for context, keyword_question_distractors in Total_List:
    if context:
        print(f"context: {context}")
        print()
        for keyword, distractors, question in keyword_question_distractors:
            print(f"Keyword: {keyword}")
            print(f"Question: {question}")
            print(f"Distractors: {distractors}")
            print()

# ###########################################

# For Training Time

### loading the model and the tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig

model_name_or_path = "facebook/bart-base" # google/flan-t5-base

# Define the quantization configuration with 4-bit
# quantization_config = BitsAndBytesConfig(bit_width=4, bnb_4bit_compute_type="torch.float16")

# Load the model with the specified quantization configuration
G_model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name_or_path,
    torch_dtype=torch.float32,
    device_map='auto',
)
G_tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model
Lora_config = LoraConfig(
    r=18,
    lora_alpha=12,
    target_modules=["q_proj", "v_proj"], # or q and v for other models
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

L_model = get_peft_model(G_model, Lora_config)
print(L_model.print_trainable_parameters())

In [ ]:
from datasets import Dataset, load_dataset

#for diff english data
en = (
    load_dataset("mou3az/Question-Answering-Generation-Choices", split="train")
    .filter(lambda example: len(example["context"]) < 951)
)
# # Split the filtered dataset into training and validation sets
en_train = en.select(range(19500))  # Select first 15,000 examples for training
en_validation = en.select(range(19500, 20901))  # Select next 2,500 examples for validation

In [ ]:
#For General data
# import ast
def create_prompt1(context, answer):
    input_text = f"Given the context '{context}' and the answer '{answer}' , what question can be asked?"
    return input_text

def create_prompt2(question):
    output_text = f"question: {question}"
    return output_text

In [ ]:
#for english data
en_train_data = en_train.map(lambda samples: G_tokenizer.encode_plus(create_prompt1(samples['context'], samples['answer']), padding=True), remove_columns=["context", "answer", "question","distractors"])
en_validation_data = en_validation.map(lambda samples: G_tokenizer.encode_plus(create_prompt1(samples['context'], samples['answer']), padding=True), remove_columns=["context", "answer", "question",'distractors'])
en_question_Tdata = en_train.map(lambda samples: G_tokenizer.encode_plus(create_prompt2(samples['question']), padding=True), remove_columns=["context", "answer", "question",'distractors'])["input_ids"]
en_question_Vdata = en_validation.map(lambda samples: G_tokenizer.encode_plus(create_prompt2(samples['question']), padding=True), remove_columns=["context", "answer", "question",'distractors'])["input_ids"]
en_train_data=en_train_data.add_column("labels", en_question_Tdata)
en_validation_data=en_validation_data.add_column("labels", en_question_Vdata)

In [ ]:
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, EarlyStoppingCallback, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
      gradient_accumulation_steps=10,
      per_device_train_batch_size=45,
      per_device_eval_batch_size=45,
      # save_steps=2,
      eval_steps=150,
      warmup_steps=150,
      logging_steps=150,
      weight_decay=0.05,
      # save_total_limit=5,
      learning_rate=3e-3,
      max_steps=3000,
      # num_train_epochs=2,
      # load_best_model_at_end=True,
      # gradient_checkpointing=True,
      lr_scheduler_type="linear",
      do_train=True,
      do_eval=True,
      # fp16=False,
      report_to="all",
      log_level="debug",
      logging_dir='./logs',
      output_dir='./outputs',
      label_names=["labels"],
      evaluation_strategy="steps",
      # metric_for_best_model="eval_loss",
    )

trainer = Seq2SeqTrainer(
    model=L_model,
    args=training_args,
    tokenizer=G_tokenizer,
    train_dataset=en_train_data,
    eval_dataset=en_validation_data,
    # callbacks=[EarlyStoppingCallback(2, 1.0)],
    data_collator=DataCollatorForSeq2Seq(G_tokenizer,label_pad_token_id=-100),
)

# Additional configuration
L_model.config.use_cache = False
torch.cuda.empty_cache()
# L_model.config.bnb_8bit_compute_type = "torch.float16"

# Start training
trainer.train()

In [ ]:
# to hugging face
model_name = ""
HUGGING_FACE_USER_NAME = ""

L_model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{model_name}", token='')

In [ ]:
# Save model checkpoint
L_model.save_pretrained("")
# Create a zip archive
!zip -r saved_model.zip VF

In [ ]:
#general question generation model
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
HUGGING_FACE_USER_NAME=''
model_name=''
peft_model_id = f"{HUGGING_FACE_USER_NAME}/{model_name}"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=False, device_map='auto')
G_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
L_model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
def generate_questions(context, answer):
    device = next(L_model.parameters()).device
    input_text = f"Given the context '{context}' and the answer '{answer}', what question can be asked?"
    encoding = G_tokenizer.encode_plus(input_text, padding=True, return_tensors="pt").to(device)

    output_tokens = L_model.generate(
        **encoding,
        early_stopping=True,
        do_sample= True,
        num_beams=5,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        max_length=256,
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2
    )
    question = G_tokenizer.decode(output_tokens[0], skip_special_tokens=True).replace("question :", "").strip()
    return question

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
from fuzzywuzzy import fuzz
from bert_score import score


def calculate_bleu_scores(references, predictions):
    return corpus_bleu([[ref.split()] for ref in references], [pred.split() for pred in predictions])

def calculate_rouge_scores(references, predictions):
    rouge = Rouge()
    rouge_scores = rouge.get_scores(predictions, references, avg=True)
    return rouge_scores

def calculate_accuracy(references, predictions):
    accuracies = [fuzz.token_sort_ratio(ref, pred) / 100.0 for ref, pred in zip(references, predictions)]
    return sum(accuracies) / len(accuracies)

def calculate_bert_score(references, predictions):
    P, R, F1 = score(predictions, references, lang='en', verbose=False)
    return F1.mean().item()

def evaluate(dataset):
    references = [sample['question'] for sample in dataset]
    predictions = [generate_questions(sample['context'], sample['answer']) for sample in dataset]  # Assuming 'generate_questions' generates model's output

    bleu_score = calculate_bleu_scores(references, predictions)
    rouge_scores = calculate_rouge_scores(references, predictions)
    accuracy = calculate_accuracy(references, predictions)
    bert_score = calculate_bert_score(references, predictions)

    print("Overall Accuracy:", accuracy)
    print("Overall BLEU Score:", bleu_score)
    print("Overall ROUGE Score:", rouge_scores)
    print("Overall BERTScore:", bert_score)

    return accuracy, bleu_score, rouge_scores, bert_score

# Assuming 'en_validation' is your dataset
accuracy, bleu_score, rouge_scores, bert_score = evaluate(en_validation)

# ###########################################

# Other Method

In [ ]:
!pip install huggingface_hub flask

In [ ]:
from flask import Flask, request, jsonify
from huggingface_hub import InferenceClient
from requests.exceptions import RequestException
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_hf_token"

app = Flask(__name__)


client = InferenceClient("mistralai/Mistral-7B-Instruct-v0.2")

def system_instructions(context):
    return f"""<s> [INST] Your are a great teacher and your task is to create 6 questions with answer and 4 choices based on the following context:\n\n{context}\n\n. Each example should be like this
    Question: ""
    Choices:
    A): ""
    B): ""
    C): ""
    D): ""
    Answer: "A or B or C or D according to the right answer"
    Explanation: ""
    \n
    [/INST]
    """

def generate_quiz(context):
    formatted_prompt = system_instructions(context)

    generate_kwargs = dict(
            temperature=0.1,
            max_new_tokens=2048,
            top_p=0.95,
            repetition_penalty=1.0,
            do_sample=True,
            seed=42,)

    try:
        response = client.text_generation(
            formatted_prompt,
            **generate_kwargs,
            stream=False,
            details=False,
            return_full_text=False,
        )
        return response

    except (RequestException, SystemExit) as e:
        return {"error": str(e)}

@app.route("/", methods=["GET", "POST"])
def generate_quiz_page():
    if request.method == "POST":
        if request.content_type == 'application/json':
            data = request.get_json()
            context = data.get("context")
            if context is None or context.strip() == "":
                return jsonify({"error": "Missing or empty 'context' parameter"}), 400
        else:
            context = request.form.get("context")
            if context is None or context.strip() == "":
                return jsonify({"error": "Missing or empty 'context' parameter"}), 400

        response = generate_quiz(context)

        if request.content_type == 'application/json':
            return jsonify(response)


        quiz_html = f"""
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Generated Quiz</title>
            <style>
                body {{
                    font-family: Arial, sans-serif;
                    background-color: #f0f0f0; /* Light grey background */
                    color: #333; /* Dark grey text color */
                    margin: 20px;
                }}
                h2 {{
                    background-color: #ffc107; /* Yellow background for heading */
                    padding: 10px;
                    border-radius: 5px;
                }}
                form {{
                    display: none; /* Hide the form after generating quiz */
                }}
                .quiz-container {{
                    background-color: #f0f0f0; /* Light grey background */
                    padding: 20px;
                    border-radius: 5px;
                    box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); /* Light shadow effect */
                    margin-bottom: 20px;
                }}
                .quiz-container pre {{
                    white-space: pre-wrap; /* Preserve line breaks in quiz response */
                }}
                .generate-link {{
                    display: inline-block; /* Make the link inline-block */
                    margin-top: 10px;
                    border: 2px solid transparent; /* Transparent border initially */
                    padding: 5px 10px; /* Padding for spacing inside the border */
                    text-decoration: none; /* Remove underline */
                    color: #ffc107; /* Yellow text color */
                    font-weight: bold; /* Bold text */
                    border-radius: 4px;
                    background-color: #333; /* Dark grey background color */
                }}
                .generate-link:hover {{
                    text-decoration: none; /* Remove underline on hover */
                    background-color: #555; /* Darker grey background color on hover */
                }}
                a {{
                    color: #ffc107; /* Yellow text color for links */
                    text-decoration: none;
                }}
                a:hover {{
                    text-decoration: underline; /* Underline on hover */
                }}
                textarea {{
                    width: 100%;
                    padding: 10px;
                    border: 1px solid #ccc;
                    border-radius: 4px;
                    resize: vertical; /* Allow vertical resizing of textarea */
                    min-height: 150px; /* Minimum height for textarea */
                }}
                input[type="submit"] {{
                    background-color: #ffc107; /* Yellow background for submit button */
                    color: #333; /* Dark grey text color */
                    border: none;
                    padding: 10px 20px;
                    cursor: pointer;
                    border-radius: 4px;
                }}
                input[type="submit"]:hover {{
                    background-color: #ffca28; /* Lighter yellow on hover */
                }}
            </style>
        </head>
        <body>
            <div class="quiz-container">
                <h2>Generated Quiz</h2>
                <pre>{response}</pre>
                <a href="/" class="generate-link">Back to generate another quiz</a>
            </div>
        </body>
        </html>
        """

        return quiz_html

    # Default GET request handling
    default_html = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Generate Quiz</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f0f0f0; /* Light grey background */
                color: #333; /* Dark grey text color */
                margin: 20px;
            }
            h2 {
                background-color: #ffc107; /* Yellow background for heading */
                padding: 10px;
                border-radius: 5px;
            }
            form {
                background-color: #fff; /* White background for form */
                padding: 20px;
                border-radius: 5px;
                box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); /* Light shadow effect */
                margin-bottom: 20px;
            }
            label {
                display: block;
                margin-bottom: 10px;
            }
            textarea {
                width: 98%;
                padding: 10px;
                border: 2px solid #ffc107; /* Yellow border */
                border-radius: 4px;
                resize: vertical; /* Allow vertical resizing of textarea */
                min-height: 150px; /* Minimum height for textarea */
                background-color: #f0f0f0; /* Light grey background */
            }
            input[type="submit"] {
                background-color: #ffc107; /* Yellow background for submit button */
                color: #333; /* Dark grey text color */
                border: none;
                padding: 10px 20px;
                cursor: pointer;
                border-radius: 4px;
            }
            input[type="submit"]:hover {
                background-color: #ffca28; /* Lighter yellow on hover */
            }
        </style>
    </head>
    <body>
        <h2>Generate Quiz</h2>
        <form action="/" method="post">
            <label for="context">Context:</label><br>
            <textarea id="context" name="context" rows="20" required></textarea><br><br>
            <input type="submit" value="Generate Quiz">
        </form>
    </body>
    </html>
    """

    return default_html

if __name__ == "__main__":
    app.run(debug=True)